In [2]:
import pandas as pd
import numpy as np
import os
import zipfile
import json
import tensorflow as tf
import torch
from transformers import BertTokenizer, BertModel
from sentence_transformers import SentenceTransformer, InputExample, losses
import faiss
from faiss import IndexFlatL2
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
tf.debugging.set_log_device_placement(True)
df = pd.read_csv('archive2/nq_small.csv')

In [4]:
df.head(10)

,answer,context,id,question,answer_start,p0,p1,alen
0,103 Hz,\n\nThe hertz (symbol: Hz) is the derived unit...,532428134682627320,1 kilohertz is equal to how many hertz?,324,-0.469617,0.347219,6
1,1/18th the size,1:18 scale diecast replicas are 1/18th the siz...,-4516564998123606940,How big is a 1 18 scale model?,32,-1.959410,1.614884,15
2,380 AD,Nicene Christianity became the state church of...,4884836952562841253,When did christianity become official religion...,98,0.222497,-0.266590,6
3,seventeenth-century,Salt and pepper is the common name for edible ...,6442551588281709780,When did salt and pepper became a pair?,352,-0.315290,0.215326,19
4,Harry S. Truman,"\n\nHarry S. Truman[b] (May 8, 1884 – December...",9214285402926650474,Who was president when world war 2 ended?,2,-0.080704,-0.262564,15
5,Call of Duty,Call of Duty is a first-person shooter video g...,-59817003176266817,What was the first call of duty game ever made?,0,0.095545,-0.257527,12
6,48 BC,The Library of Alexandria was one of the large...,2093732342481935035,When did the library of alexandria burn down?,1015,-0.536527,0.298526,5
7,employees injured in the course of employment,Workers' compensation is a form of insurance p...,528868946182004081,When does an employee qualify for workers comp...,96,-0.557063,0.329259,45
8,universal motors,The universal motor is so named because it is ...,7651063123716449392,What type of motor used in vacuum cleaner?,1330,-0.770690,0.507451,16
9,the Indian painter Abanindranath Tagore,\n\nBharat Mata is a work painted by the India...,-5586968952968112296,Who was the painter of this famous painting bh...,35,0.045844,-0.234746,39


In [ ]:
import torch
from transformers import BertModel

class DualEncoderModel(torch.nn.Module):
    def __init__(self, question_encoder, passage_encoder):
        super(DualEncoderModel, self).__init__()
        self.question_encoder = question_encoder
        self.passage_encoder = passage_encoder

    def forward(self, question_ids, question_mask, passage_ids, passage_mask):
        question_outputs = self.question_encoder(input_ids=question_ids, attention_mask=question_mask)
        passage_outputs = self.passage_encoder(input_ids=passage_ids, attention_mask=passage_mask)
        
        # Pooling the output embeddings of the [CLS] token
        question_pooler_output = question_outputs.pooler_output
        passage_pooler_output = passage_outputs.pooler_output
        return question_pooler_output, passage_pooler_output


In [ ]:
model_name = "bert-base-uncased"  # make sure to use the same model name
question_encoder = BertModel.from_pretrained(model_name)
passage_encoder = BertModel.from_pretrained(model_name)
model = DualEncoderModel(question_encoder, passage_encoder)

In [ ]:
model_path = 'dual_encoder_model.pth'
checkpoint = torch.load(model_path)

model.question_encoder.load_state_dict(checkpoint['question_encoder_state_dict'])
model.passage_encoder.load_state_dict(checkpoint['passage_encoder_state_dict'])

optimizer = torch.optim.Adam(model.parameters()) 
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])


In [8]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Encode passages and questions
passage_encodings = []
question_encodings = []

for index, row in df.iterrows():
    passage_tokens = tokenizer.encode(row['context'], add_special_tokens=True, max_length=64, truncation=True, padding='max_length')
    question_tokens = tokenizer.encode(row['question'], add_special_tokens=True, max_length=64, truncation=True, padding='max_length')

    # with torch.no_grad():
    #     passage_output = p_model(torch.tensor([passage_tokens]))[0]
    #     question_output = q_model(torch.tensor([question_tokens]))[0]

    question_embedding,passage_embedding=model(question_tokens['input_ids'].squeeze(), question_tokens['attention_mask'].squeeze(), passage_tokens['input_ids'].squeeze(), passage_tokens['attention_mask'].squeeze())
    passage_encodings.append(passage_embedding)
    question_encodings.append(question_embedding)


passage_encodings = torch.stack(passage_encodings).numpy()
question_encodings = torch.stack(question_encodings).numpy()

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing 

Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /j

Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:loc

Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /j

Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0


...

Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:loc

Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /j

Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:loc

Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /j

All TF 2.0 model weights were used when initializing BertModel.

All the weights of BertModel were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertModel for predictions without further training.
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


In [9]:
print("Shape of passage_encodings:", passage_encodings.shape)
print("Data type of passage_encodings:", passage_encodings.dtype)

passage_encodings = passage_encodings.astype('float32')
passage_encodings = passage_encodings.reshape(passage_encodings.shape[0], -1)

d = passage_encodings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(passage_encodings)

accuracy_matrix = []

for _, row in df.iterrows():
    question_tokens = tokenizer.encode(row['question'], add_special_tokens=True, max_length=64, truncation=True, padding='max_length')

    with torch.no_grad():
        question_output = q_model(torch.tensor([question_tokens]))[0]

    question_encoding = question_output.detach().numpy().reshape(1, -1)

    D, I = index.search(question_encoding, k=1)

    top_passage_context = df.iloc[I[0][0]]['context']
    ground_truth_passage = row['context']

    sim_score = cosine_similarity(question_output.detach().numpy().reshape(1, -1), passage_encodings[I[0][0]].reshape(1, -1))

    ground_truth_passage_encoding = tokenizer.encode(ground_truth_passage, add_special_tokens=True, max_length=64, truncation=True, padding='max_length')
    ground_truth_passage_encoding,_ = model(question_tokens['input_ids'].squeeze(), question_tokens['attention_mask'].squeeze(), question_tokens['input_ids'].squeeze(), question_tokens['attention_mask'].squeeze())

    sim_score_ground_truth = cosine_similarity(passage_encodings[I[0][0]].reshape(1, -1), ground_truth_passage_encoding.reshape(1, -1))

    accuracy_matrix.append((row['question'], top_passage_context, ground_truth_passage, sim_score, sim_score_ground_truth))


Shape of passage_encodings: (10658, 768)
Data type of passage_encodings: float32


In [ ]:
max_score = -1
best_passage = None

for item in accuracy_matrix:
    _, _, passage, _, sim_score_ground_truth = item
    if sim_score_ground_truth > max_score:
        max_score = sim_score_ground_truth
        best_passage = passage

print("Highest similarity score:", max_score)
print("Corresponding passage:", best_passage)


Highest similarity score: [[1.]]
Corresponding passage: Riverdale High School can refer to:



In [ ]:
max_score_ground_truth = -1
best_row = None

for item in accuracy_matrix:
    question, top_passage_context, ground_truth_passage, sim_score, sim_score_ground_truth = item
    if sim_score_ground_truth > max_score_ground_truth:
        max_score_ground_truth = sim_score_ground_truth
        best_row = item

print("Highest similarity score (ground truth):", max_score_ground_truth)
print("Corresponding row details:")
print("Question:", best_row[0])
print("Top Passage Context:", best_row[1])
print("Ground Truth Passage:", best_row[2])
print("Similarity Score (question and passage):", best_row[3])
print("Similarity Score (ground truth):", best_row[4])


Highest similarity score (ground truth): [[1.]]
Corresponding row details:
Question: What is the high school in riverdale called?
Top Passage Context: Riverdale High School can refer to:

Ground Truth Passage: Riverdale High School can refer to:

Similarity Score (question and passage): [[0.8720614]]
Similarity Score (ground truth): [[1.]]


In [ ]:
min_score_ground_truth = float('inf')
worst_row = None

for item in accuracy_matrix:
    question, top_passage_context, ground_truth_passage, sim_score, sim_score_ground_truth = item
    if sim_score_ground_truth < min_score_ground_truth:
        min_score_ground_truth = sim_score_ground_truth
        worst_row = item

print("Least similarity score (ground truth):", min_score_ground_truth)
print("Corresponding row details:")
print("Question:", worst_row[0])
print("Top Passage Context:", worst_row[1])
print("Ground Truth Passage:", worst_row[2])
print("Similarity Score (question and passage):", worst_row[3])
print("Similarity Score (ground truth):", worst_row[4])


Least similarity score (ground truth): [[0.03175506]]
Corresponding row details:
Question: Who won the 1980 men's wimbledon final?
Top Passage Context: Riverdale High School can refer to:

Ground Truth Passage: 

The 1980 Wimbledon Men's Singles final was the championship tennis match of the Men's Singles tournament at the 1980 Wimbledon Championships. In the final, Björn Borg defeated John McEnroe 1–6, 7–5, 6–3, 6–7(16–18), 8–6 to win the match. It was regarded at the time as the greatest match ever played. It would be another 28 years before the 2008 Wimbledon men's singles final between Roger Federer and Rafael Nadal would rival the match as the greatest ever played.

The match was eagerly awaited as not only was it a clash between the top two seeds, it was also a meeting of opposites: 

Borg was the Iceman, a ruthless baseliner who used his racket to probe other players' weaknesses like a surgeon wielding a scalpel. McEnroe was the Brat, a modern genius in the old-fashioned craft o

In [ ]:
# Initialize a dictionary to store the counts for each range
sim_score_counts = {i / 10: 0 for i in range(11)}

# Iterate through the accuracy matrix
for _, _, _, _, sim_score_ground_truth in accuracy_matrix:
    for key in sim_score_counts.keys():
        if key <= sim_score_ground_truth < key + 0.1:
            sim_score_counts[key] += 1


for key, value in sim_score_counts.items():
    print(f"Number of questions with sim_score_ground_truth between {key:.1f} and {key + 0.1:.1f}: {value}")


Number of questions with sim_score_ground_truth between 0.0 and 0.1: 1057
Number of questions with sim_score_ground_truth between 0.1 and 0.2: 9501
Number of questions with sim_score_ground_truth between 0.2 and 0.3: 89
Number of questions with sim_score_ground_truth between 0.3 and 0.4: 3
Number of questions with sim_score_ground_truth between 0.4 and 0.5: 3
Number of questions with sim_score_ground_truth between 0.5 and 0.6: 1
Number of questions with sim_score_ground_truth between 0.6 and 0.7: 1
Number of questions with sim_score_ground_truth between 0.7 and 0.8: 1
Number of questions with sim_score_ground_truth between 0.8 and 0.9: 1
Number of questions with sim_score_ground_truth between 0.9 and 1.0: 0
Number of questions with sim_score_ground_truth between 1.0 and 1.1: 1


## Reader

In [10]:
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import pandas as pd

# Load pre-trained tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model1 = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
# Function to extract top answers from possible start and end positions
def get_top_answers(possible_starts, possible_ends, input_ids):
    answers = []
    for start, end in zip(possible_starts, possible_ends):
        # +1 for end to include the end token
        answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[start:end + 1]))
        answers.append(answer)
    return answers


In [12]:
def filter_irrelevant_answers(answers, question):
    relevant_answers = []
    question_tokens = set(tokenizer.tokenize(question.lower()))
    for ans in answers:
        answer_tokens = set(tokenizer.tokenize(ans.lower()))
        if answer_tokens.intersection(question_tokens):
            relevant_answers.append(ans)
    return relevant_answers[:3]

def answer_question(question, context, topN, chunk_size=200):
    question_tokens = tokenizer.encode(question, add_special_tokens=False, return_tensors="pt")
    context_tokens = tokenizer.encode(context, add_special_tokens=False, return_tensors="pt")
    num_chunks = len(context_tokens[0]) // chunk_size + 1
    all_answers = []

    for i in range(num_chunks):
        start_index = i * chunk_size
        end_index = min((i + 1) * chunk_size, len(context_tokens[0]))
        input_ids = torch.cat([question_tokens, context_tokens[:, start_index:end_index]], dim=1)
        model_out = model1(input_ids)
        answer_start_scores = model_out["start_logits"]
        answer_end_scores = model_out["end_logits"]
        possible_starts = np.argsort(answer_start_scores.cpu().detach().numpy()).flatten()[::-1][:topN]
        possible_ends = np.argsort(answer_end_scores.cpu().detach().numpy()).flatten()[::-1][:topN]
        answer_start = torch.argmax(answer_start_scores)
        answer_end = torch.argmax(answer_end_scores) + 1
        answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[0][answer_start:answer_end]))
        answers = get_top_answers(possible_starts, possible_ends, input_ids[0])
        relevant_answers = filter_irrelevant_answers(answers, question)
        all_answers.extend(relevant_answers)

    return {
        "answers": all_answers
    }


In [26]:
limit = 0
for _, row in df.iterrows():
    limit += 1
    if limit == 11:
        break

    with torch.no_grad():
        question_tokens = tokenizer.encode(row['question'], add_special_tokens=True, max_length=64, truncation=True, padding='max_length')
        question_output,_ = model(question_tokens['input_ids'].squeeze(), question_tokens['attention_mask'].squeeze(), question_tokens['input_ids'].squeeze(), question_tokens['attention_mask'].squeeze())

    question_encoding = question_output.detach().numpy().reshape(1, -1)
    D, I = index.search(question_encoding, k=1)
    top_passage_context = df.iloc[I[0][0]]['context']
    ground_truth_passage = row['context']
    sim_score = cosine_similarity(question_output.detach().numpy().reshape(1, -1), passage_encodings[I[0][0]].reshape(1, -1))
    ground_truth_passage_encoding = tokenizer.encode(ground_truth_passage, add_special_tokens=True, max_length=64, truncation=True, padding='max_length')
    ground_truth_embedding = q_model.encode(ground_truth_passage, convert_to_tensor=True)
    sim_score_ground_truth = cosine_similarity(passage_encodings[I[0][0]].reshape(1, -1), ground_truth_embedding.numpy().reshape(1, -1))
    q = row["question"]
    t = top_passage_context
    answer_map = answer_question(str(q), str(t), 5)
    print("Question:", q)
    print("Text:", t)
    print("Answers:")
    for indexa, ans in enumerate(answer_map["answers"][:3]):
        if len(ans) > 0:
            print(f"{indexa+1}. {ans}")


Question: 1 kilohertz is equal to how many hertz?
Text: Milli- (symbol m) is a unit prefix in the metric system denoting a factor of one thousandth (10−3).[1] Proposed in 1793[2] and adopted in 1795, the prefix comes from the Latin mille, meaning "one thousand" (the Latin plural is milia). Since 1960, the prefix is part of the International System of Units (SI).
Example:
Answers:


Token indices sequence length is longer than the specified maximum sequence length for this model (2401 > 512). Running this sequence through the model will result in indexing errors


Question: How big is a 1 18 scale model?
Text: The parsec (symbol: pc) is a unit of length used to measure large distances to astronomical objects outside the Solar System. A parsec is defined as the distance at which one astronomical unit subtends an angle of one arcsecond,[1] which corresponds to 7005648000000000000♠648000/π astronomical units. One parsec is equal to about 3.26 light-years (30 trillion km or 19 trillion miles) in length. The nearest star, Proxima Centauri, is about 1.3 parsecs (4.2 light-years) from the Sun.[2] Most of the stars visible to the unaided eye in the night sky are within 500 parsecs of the Sun.[citation needed]

The parsec unit was probably first suggested in 1913 by the British astronomer Herbert Hall Turner.[3] Named as a portmanteau of the parallax of one arcsecond, it was defined so as to make calculations of astronomical distances quick and easy for astronomers from only their raw observational data. Partly for this reason, it is the unit preferred i

##### 